In [1]:
import numpy as np
from pathlib import Path
import re


In [2]:
with Path("../12.in").open() as file:
    data = file.read().splitlines()


In [3]:
testdata = """\
???.### 1,1,3
.??..??...?##. 1,1,3
?#?#?#?#?#?#?#? 1,3,1,6
????.#...#... 4,1,1
????.######..#####. 1,6,5
?###???????? 3,2,1
""".splitlines()


In [4]:
arrangement_test = """\
?###???????? 3,2,1"""
arrangement_result = """\
.###.##.#...
.###.##..#..
.###.##...#.
.###.##....#
.###..##.#..
.###..##..#.
.###..##...#
.###...##.#.
.###...##..#
.###....##.#""".splitlines()


In [5]:
# %%timeit

list(filter(bool, map(len, arrangement_result[0].split("."))))


[3, 2, 1]

In [6]:
# %%timeit

list(map(len, re.findall(r"(?:#)+", arrangement_result[0])))


[3, 2, 1]

In [7]:
# test_re = re.compile(r"(?:#)+")

# %timeit list(map(len, test_re.findall(arrangement_result[0])))


## Part I

- **Input**:
  - Springs (e.g. `#.#.###`)
    - `.` are operational springs
    - `#` are broken springs
    - `?` are springs of unknown status
  - "Checksum" (e.g. `1,1,3`)
    - Contiguous Groups of damaged springs
- **Question**:
  - How many different arrangements produce the same checksum?
- **Approach**:
  - Write a function that checks if an arrangement is valid
  - Calculate how many unknown springs need to be broken to pass the checksum
  - Split off parts of the row that are already valid, including the bordering springs
    - **`?###?`**`??????? `**`3`**`,2,1"""`
  - Build all combinations with the total count
    - Iterate each position and try all combinations of broken springs 


In [8]:
def parse(lines):
    rows = [line.split(" ")[0] for line in lines]
    hints = [list(map(int, line.split(" ")[1].split(","))) for line in lines]
    return rows, hints

parse(testdata)


(['???.###',
  '.??..??...?##.',
  '?#?#?#?#?#?#?#?',
  '????.#...#...',
  '????.######..#####.',
  '?###????????'],
 [[1, 1, 3], [1, 1, 3], [1, 3, 1, 6], [4, 1, 1], [1, 6, 5], [3, 2, 1]])

In [9]:
def is_valid_arrangement(springs, hints):
    found_hints = list(filter(bool, map(len, springs.split("."))))
    return found_hints == hints

for line in arrangement_result:
    print(is_valid_arrangement(line, [3, 2, 1]))


True
True
True
True
True
True
True
True
True
True


In [10]:
def generate_all_arrangements(springs, hints):
    arrangements = [springs[0]] if springs[0] in ".#" else [".", "#"]
    for char in springs[1:]:
        if char in ".#":
            for i in range(len(arrangements)):
                arrangements[i] += char
        else:
            count = len(arrangements)
            for i in range(count):
                arrangements.append(arrangements[i] + "#")
                arrangements[i] += "."

    # print(arrangements)
    return list(filter(lambda arr: is_valid_arrangement(arr, hints), arrangements))


In [11]:
%load_ext line_profiler


In [12]:

def solve(rows, checks):
    num_valid_arrs = 0
    for springs, groups in zip(rows, checks):
        valid_arrs = generate_all_arrangements(springs, groups)
        # print(f"{len(valid_arrs):>3} valid arrangements for {springs:<30} {groups}")
        num_valid_arrs += len(valid_arrs)

    print(f"Part 1: {num_valid_arrs}")

solve(*parse(testdata))


Part 1: 21


## Part II

- Each input is "folded" and needs to be multiplied 5 times
  - `.# 1` becomes `.#?.#?.#?.#?.# 1,1,1,1,1`
  - `?` separate each multiplication

In [13]:
def unfold(rows, hints, mult=5):
    rows = tuple(["?".join([row,] * mult) for row in rows])
    hints = tuple([tuple(check * mult) for check in hints])
    return rows, hints

unfold(*parse(testdata))


(('???.###????.###????.###????.###????.###',
  '.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.',
  '?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#?',
  '????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#...',
  '????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.',
  '?###??????????###??????????###??????????###??????????###????????'),
 ((1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3),
  (1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3),
  (1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6),
  (4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1),
  (1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5),
  (3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1)))

In [14]:
import functools


In [15]:
@functools.lru_cache(maxsize=None)
def get_valid_arrangements(springs: str, hints: tuple[int], cont_count: int=0):
    """Recursively count valid arrangements. Heavily relies on caching.

    Args:
        springs (str): String of springs (".", "#", "?").
        hints (list): List of hints (int).
        cont_count (int): Number of contiguous broken springs.
    """

    if not springs:
        # No springs left, check if we have any hints left
        if len(hints) == 1 and hints[0] == cont_count:
            # Last hint is satisfied -> valid
            return 1
        elif len(hints) == 0 and cont_count == 0:
            # End reached and no hints left -> valid
            return 1
        else:
            # End reached but hints left -> invalid
            return 0

    spring, new_springs = springs[0], springs[1:]
    hint, new_hints = (hints[0], hints[1:]) if hints else (0, tuple())

    if spring == "?":
        # Try both options, keeping the current hint and counter.
        # Add their results.
        return (get_valid_arrangements("." + new_springs, hints, cont_count)
                + get_valid_arrangements("#" + new_springs, hints, cont_count))

    elif spring == "#":
        if cont_count > hint:
            # Found more contiguous broken springs than allowed -> invalid
            return 0
        else:
            # Increase the counter and recurse with the same hints
            return get_valid_arrangements(new_springs, hints, cont_count + 1)

    elif spring == ".":
        if cont_count == 0:
            # No contiguous broken springs, just passing through.
            # Keep the current hints and counter (which is 0 anyway).
            return get_valid_arrangements(new_springs, hints, cont_count)
        elif cont_count == hint:
            # All contiguous broken springs of the current hint are satisfied.
            # Continue with the next hint and reset the counter.
            return get_valid_arrangements(new_springs, new_hints, 0)
        else:
            # Contiguous broken springs interrupted -> invalid
            return 0


In [16]:
get_valid_arrangements.cache_clear()

total = 0
for row, check in zip(*unfold(*parse(testdata))):
    num_valid = get_valid_arrangements(row, check)
    print(f"{row:<30} {check} -> {num_valid}")
    total += num_valid

print(f"Part II: {total}")

print(get_valid_arrangements.cache_info())


???.###????.###????.###????.###????.### (1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3) -> 1
.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##.?.??..??...?##. (1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3, 1, 1, 3) -> 16384
?#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#???#?#?#?#?#?#?#? (1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6, 1, 3, 1, 6) -> 1
????.#...#...?????.#...#...?????.#...#...?????.#...#...?????.#...#... (4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1, 4, 1, 1) -> 16
????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####.?????.######..#####. (1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5, 1, 6, 5) -> 2500
?###??????????###??????????###??????????###??????????###???????? (3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1, 3, 2, 1) -> 506250
Part II: 525152
CacheInfo(hits=114, misses=2519, maxsize=None, currsize=2519)


In [17]:
get_valid_arrangements.cache_clear()

total = 0
for row, check in zip(*unfold(*parse(data))):
    num_valid = get_valid_arrangements(row, check)
    total += num_valid

print(f"Part II: {total}")

print(get_valid_arrangements.cache_info())


Part II: 83317216247365
CacheInfo(hits=127435, misses=2092498, maxsize=None, currsize=2092498)
